## Reading Dataset

In [23]:
import pandas as pd
dataset=pd.read_csv("OnlineNewsPopularity.csv")

In [24]:
import numpy as np
np.median(dataset['shares'])

1400.0

In [28]:
dataset['popularity'] = [4 if x >= 1400 else 2 for x in dataset['shares']]

In [34]:
dataset.head() #4 is popular, 2 is unpopular

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,popularity
0,http://mashable.com/2013/01/07/amazon-instant-...,731,12,219,0.663594,1.0,0.815385,4,2,1,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,2
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731,9,255,0.604743,1.0,0.791946,3,1,1,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,2
2,http://mashable.com/2013/01/07/apple-40-billio...,731,9,211,0.575130,1.0,0.663866,3,1,1,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,4
3,http://mashable.com/2013/01/07/astronaut-notre...,731,9,531,0.503788,1.0,0.665635,9,0,1,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,2
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.415646,1.0,0.540890,19,19,20,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,2


In [35]:
X = dataset.iloc[:,2:-2].values
X

array([[ 1.20000000e+01,  2.19000000e+02,  6.63594467e-01, ...,
        -1.87500000e-01,  0.00000000e+00,  1.87500000e-01],
       [ 9.00000000e+00,  2.55000000e+02,  6.04743081e-01, ...,
         0.00000000e+00,  5.00000000e-01,  0.00000000e+00],
       [ 9.00000000e+00,  2.11000000e+02,  5.75129531e-01, ...,
         0.00000000e+00,  5.00000000e-01,  0.00000000e+00],
       ...,
       [ 1.00000000e+01,  4.42000000e+02,  5.16355139e-01, ...,
         1.36363636e-01,  4.54545450e-02,  1.36363636e-01],
       [ 6.00000000e+00,  6.82000000e+02,  5.39493293e-01, ...,
         0.00000000e+00,  5.00000000e-01,  0.00000000e+00],
       [ 1.00000000e+01,  1.57000000e+02,  7.01986750e-01, ...,
         2.50000000e-01,  1.66666667e-01,  2.50000000e-01]])

In [37]:
Y = dataset.iloc[:, -1].values
Y

array([2, 2, 4, ..., 4, 2, 2], dtype=int64)

In [38]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [39]:
#Fitting Gradient Boosting to Training set
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
classifier.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [43]:
#Predicting the Test set results
Y_pred = classifier.predict(X_test)
Y_pred

array([2, 4, 4, ..., 4, 2, 4], dtype=int64)

### According to the confusion matrix gotten by comparing the predicted classes to the test classes, our model got 5277 correct classifications out of 7929 test classifications. This gives an accuracy of 67%.

In [42]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
cm

array([[2281, 1458],
       [1166, 3024]], dtype=int64)

In [45]:
def precision(label,confusion_matrix):
    col = confusion_matrix[:,label]
    return confusion_matrix[label,label] / col.sum()

def recall(label,confusion_matrix):
    row = confusion_matrix[label,:]
    return confusion_matrix[label,label] / row.sum()

print("label precision recall")
for label in range(0,2):
    print(f"{label : 5d} {precision(label,cm) : 9.3f} {recall(label,cm) : 6.3f}")

label precision recall
    0     0.662  0.610
    1     0.675  0.722
